# 加载环境

In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"

try:
    import dgl
    installed = True
except ImportError:
    installed = False
print("DGL installed!" if installed else "Failed to install DGL!")

import os
from random import random

import numpy as np
import pandas as pd


import random 
from tqdm import tqdm

from GCN import  GCN

import dgl
import dgl.nn as dglnn

import torch
import torch.nn as nn
import torch.nn.functional as F

/Users/tangting/opt/anaconda3/envs/py39dgl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DGL installed!


# 仿真数据

In [15]:

# 节点
vertex_count = 1000
origin_ids = []
int_fetaures = []
float_fetaures = []
node_label_0 = []
node_label_1 = []
for i in range(vertex_count):
    origin_id = random.randint(1000000,100000000)
    if origin_id not in origin_ids:
        origin_ids.append(origin_id)
        int_fetaures.append([random.randint(10,100),random.randint(200,300)])
        float_fetaures.append([random.random()*10,random.random()*2000])
        node_label_0.append(int(random.random()*2))    
        node_label_1.append(int(random.random()*2))



# vertext_tb = pd.DataFrame(data=[origin_ids,int_fetaures,float_fetaures],columns=["origin_id","int_fetaures","float_fetaures"])
vertext_tb = pd.DataFrame({"origin_id" : origin_ids, \
                            "int_fetaures": int_fetaures, \
                            "float_fetaures" :float_fetaures,
                            "node_label_0":node_label_0,\
                           "node_label_1":node_label_1     })
vertext_tb["id"] = vertext_tb.index
vertext_tb.to_csv("data/vertex_data.csv",index=False,header=True)
vertext_tb

,origin_id,int_fetaures,float_fetaures,node_label_0,node_label_1,id
0,35081977,"[28, 211]","[8.801850035868103, 14.126212864719268]",1,1,0
1,50671236,"[10, 277]","[1.6100933521302119, 145.43665551281282]",1,0,1
2,47890196,"[90, 206]","[0.8585547459186338, 521.4852659843707]",1,0,2
3,43006567,"[92, 264]","[2.089113347203664, 273.0179886980648]",0,0,3
4,49343354,"[95, 223]","[7.3813898475883555, 1773.4458706289288]",0,1,4
...,...,...,...,...,...,...
995,62930113,"[41, 296]","[4.856071898587158, 1204.131136066832]",1,1,995
996,79809437,"[44, 212]","[9.81411871969672, 27.76764636788598]",1,0,996
997,67024796,"[23, 254]","[4.425254092054488, 1052.7992688689696]",0,1,997
998,97344701,"[41, 229]","[4.609263212688553, 874.9417087062473]",0,1,998


In [19]:
vertex_count = len(origin_ids)
edge_count = vertex_count * 10
edge_src_ids = []
edge_dst_ids = []
edge_types = []
for i in range(edge_count):
    src_rand_id = random.randint(0,vertex_count-1)
    dst_rand_id = random.randint(0,vertex_count-1)
    if src_rand_id != dst_rand_id:
        src_id = origin_ids[src_rand_id]
        edge_src_ids.append(src_id)
        dst_id = origin_ids[dst_rand_id]
        edge_dst_ids.append(dst_id)
        edge_types.append(random.randint(0,5))

# vertext_tb = pd.DataFrame(data=[origin_ids,int_fetaures,float_fetaures],columns=["origin_id","int_fetaures","float_fetaures"])
edge_tb = pd.DataFrame({"src_origin_id" : edge_src_ids,"dst_origin_id": edge_dst_ids,"edge_type" :edge_types})


In [20]:
edge_tb = pd.merge(edge_tb,vertext_tb[["origin_id","id"]],left_on="src_origin_id",right_on="origin_id",how="left") .rename(columns={"id":"src_id"}).drop(columns=["origin_id"])
edge_tb = pd.merge(edge_tb,vertext_tb[["origin_id","id"]],left_on="dst_origin_id",right_on="origin_id",how="left") .rename(columns={"id":"dst_id"}).drop(columns=["origin_id"])

edge_tb.to_csv("data/edge_data.csv",index=False,header=True)

# 加载成图

In [18]:
vertext_tb = pd.read_csv( "data/vertex_data.csv",converters={'int_fetaures': lambda x: np.array(x.strip('[]').split(','), dtype=np.int32),
                                                            'float_fetaures': lambda x: np.array(x.strip('[]').split(','), dtype=np.float)})

edge_tb = pd.read_csv("data/edge_data.csv")

/var/folders/23/8rnl11914v1_gmqkg83jtt8h0000gn/T/ipykernel_29113/1470418288.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'float_fetaures': lambda x: np.array(x.strip('[]').split(','), dtype=np.float)})


In [19]:
def _preprocess_features(feature):
    return feature

In [20]:
# 加载成图

u,v = edge_tb["src_id"].tolist(),edge_tb["dst_id"].tolist()
g = dgl.graph((u, v))

In [22]:
g.ndata['int_feat'] = torch.tensor(vertext_tb["int_fetaures"])
g.ndata['float_feat'] = torch.tensor(_preprocess_features(vertext_tb["float_fetaures"]))

g.ndata['pre_label'] = torch.tensor(vertext_tb["node_label_0"],dtype=torch.float)
g.ndata['label'] = torch.tensor(vertext_tb["node_label_1"],dtype=torch.float)

g.edata['edge_type'] = torch.tensor(edge_tb["edge_type"],dtype=torch.float)

/var/folders/23/8rnl11914v1_gmqkg83jtt8h0000gn/T/ipykernel_29113/845912268.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:204.)
  g.ndata['int_feat'] = torch.tensor(vertext_tb["int_fetaures"])


In [9]:
g.ndata["train_mask"] = torch.randint(0,2,[g.ndata['label'].shape[0],1])

g.ndata["val_mask"] = torch.randint(0,2,[g.ndata['label'].shape[0],1])
g.ndata["test_mask"] = torch.randint(0,2,[g.ndata['label'].shape[0],1])

g.ndata["feat"] = torch.concat([g.ndata['int_feat'],g.ndata['float_feat'],torch.reshape(g.ndata['pre_label'],[-1,1])],axis =1)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
g = g.int().to(device)
features = g.ndata["feat"]
labels = g.ndata["label"]
masks = g.ndata["train_mask"], g.ndata["val_mask"], g.ndata["test_mask"]

# create GCN model
in_size = features.shape[1]
out_size = 1
model = GCN(in_size, 16, out_size).to(device)

In [12]:
def evaluate(g, features, labels, mask, model, active_fuc):
    model.eval()
    with torch.no_grad():
        logits = active_fuc(model(g, features))
        y_pred_prob = logits[mask].squeeze().detach().numpy()
        y_pred_label =  (y_pred_prob >= 0.5).astype(int)
        labels = labels[mask]

        accuracy = (y_pred_label == labels.numpy().reshape(-1)).mean()
        return accuracy


def train(g, features, labels, masks, model):
    # define train/val samples, loss function and optimizer
    train_mask = masks[0]
    val_mask = masks[1]
    loss_fcn = nn.MSELoss()
    sigmoid = nn.Sigmoid()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-4)

    # training loop
    for epoch in range(20):
        model.train()
        logits = model(g, features)
        pred = sigmoid(logits)
        loss = loss_fcn(pred[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = evaluate(g, features, labels, val_mask, model, sigmoid)
        print(
            "Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f} ".format(epoch, loss.item(), acc)
        )

# model training
print("Training...")
train(g, features, labels, masks, model)

# test the model
print("Testing...")
acc = evaluate(g, features, labels, masks[2], model, nn.Sigmoid())
print("Test accuracy {:.4f}".format(acc))

Training...
Epoch 00000 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00001 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00002 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00003 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00004 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00005 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00006 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00007 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00008 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00009 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00010 | Loss 1.0000 | Accuracy 0.0000 


/Users/tangting/opt/anaconda3/envs/py39dgl/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1000, 1])) that is different to the input size (torch.Size([1000, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/tangting/opt/anaconda3/envs/py39dgl/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 00011 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00012 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00013 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00014 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00015 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00016 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00017 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00018 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00019 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00020 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00021 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00022 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00023 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00024 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00025 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00026 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00027 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00028 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00029 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00030 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00031 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00032 | Loss 1.0000 | Accuracy 0.0000 
Epoch 0003